In [ ]:
# Load model directly
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

In [ ]:
import pandas as pd
import numpy as np
import torch

	
df = pd.read_csv("../mango/train.csv", sep=";")

import pandas as pd
import numpy as np

def combine_attributes_jerarquica(row):
    """
    Combina atributos textuales en formato jerárquico
    Máxima relevancia: categoría/arquetipo → atributos físicos → contexto
    """
    # Definir columnas por nivel de importancia (excluyendo numéricas)
    texto_cols = [
        'aggregated_family', 'family', 'category',
        'fabric', 'color_name',
        'length_type', 'silhouette_type', 'waist_type',
        'neck_lapel_type', 'sleeve_length_type', 
        'heel_shape_type', 'toecap_type',
        'woven_structure', 'knit_structure',
        'print_type', 'archetype', 'moment'
    ]
    
    # Extraer valores no nulos
    valores = []
    for col in texto_cols:
        val = row[col]
        # Ignorar NaN, None, y strings vacíos
        if pd.notna(val) and str(val).strip():
            valores.append(str(val).strip())
    valores.append(f'has plus sizes {row["has_plus_sizes"]}')
    # Unir con separadores claros
    return ' , '.join(valores)  # Usa ' | ' para separación visual clara


# Cargar datos (ajusta separador si es necesario)

# ELEGIR UNA ESTRATEGIA
# Para producción general: usa la jerárquica
df['combined_text'] = df.apply(combine_attributes_jerarquica, axis=1)

# Ver ejemplo
print("Ejemplo combinado:\n", df['combined_text'].iloc[0])

# Generar embeddings
embeddings = model.encode(
    df['combined_text'].tolist(),
    convert_to_tensor=True,
    show_progress_bar=True  # Para ver el progreso
)

print(f"\nEmbeddings generados: {embeddings.shape}")

# Guardar
torch.save(embeddings, 'embeddings_prendas.pt')
df[['ID', 'combined_text']].to_csv('textos_combinados.csv', index=False)

In [ ]:
embeddings_tensor = torch.load('embeddings_prendas.pt')
print(f"✓ Embeddings cargados: {embeddings_tensor.shape}")

In [ ]:
import pandas as pd
import torch

# ============================================
# 1. CARGAR LOS ARCHIVOS QUE YA TIENES
# ============================================


# Cargar tensor de embeddings (lo que guardaste con torch.save)


# Cargar DataFrame original (tu CSV completo)
df_original = pd.read_csv('../mango/train.csv', sep=';')
print(f"✓ DataFrame original: {df_original.shape}")

# Cargar referencia de IDs (opcional pero recomendado para verificar)
df_textos = pd.read_csv('textos_combinados.csv')
print(f"✓ DataFrame de referencia: {df_textos.shape}")


# ============================================
# 2. CREAR DataFrame con embeddings e IDs
# ============================================

# Convertir tensor a lista de listas (cada embedding es una lista)
embeddings_lista = embeddings_tensor.cpu().numpy().tolist()

# Crear DataFrame temporal con ID y embedding
embeddings_df = pd.DataFrame({
    'ID': df_textos['ID'],  # Usamos los IDs del archivo de referencia
    'embedding_tabla': embeddings_lista
})

print(f"\n✓ DataFrame de embeddings creado: {embeddings_df.shape}")


# ============================================
# 3. FUSIONAR CON EL DATAFRAME ORIGINAL (MERGE POR ID)
# ============================================

# Esto es lo MÁS IMPORTANTE: merge por ID asegura alineación perfecta
df_final = df_original.merge(embeddings_df, on='ID', how='left')

# Verificar que la fusión fue exitosa
num_sin_embedding = df_final['embedding_tabla'].isna().sum()
if num_sin_embedding > 0:
    print(f"⚠️ ALERTA: {num_sin_embedding} prendas no tienen embedding")
    print("IDs sin embedding:", df_final[df_final['embedding_tabla'].isna()]['ID'].tolist())
else:
    print("\n✅ Fusión exitosa: todos los embeddings se unieron correctamente")


# ============================================
# 4. VERIFICAR INTEGRIDAD
# ============================================

print("\n🔍 Verificación final:")
print(f"   - Filas en DataFrame final: {len(df_final):,}")
print(f"   - Columnas nuevas: {len(df_original.columns)} originales + 1 (embedding) = {len(df_final.columns)}")
print(f"   - Tamaño del embedding: {len(df_final.iloc[0]['embedding_tabla'])} dimensiones")
print(f"   - Tipo de dato: {type(df_final.iloc[0]['embedding_tabla'])}")

# Verificar que el embedding es válido (norma L2 ≈ 1.0)
primer_embedding = torch.tensor(df_final.iloc[0]['embedding_tabla'])
norma = torch.norm(primer_embedding).item()
print(f"   - Norma L2 del primer embedding: {norma:.6f} (debe ser ≈ 1.0)")


# ============================================
# 5. GUARDAR RESULTADO FINAL
# ============================================

print("\n💾 Guardando archivo final...")

# OPCIÓN A: Pickle (recomendada - preserva objetos Python)
df_final.to_pickle('prendas_con_embeddings_final.pkl')
print("   ✓ Guardado como 'prendas_con_embeddings_final.pkl'")

# OPCIÓN B: Parquet (más eficiente, compatible con Big Data)
df_final.to_parquet('prendas_con_embeddings_final.parquet', index=False)
print("   ✓ Guardado como 'prendas_con_embeddings_final.parquet'")

# OPCIÓN C: CSV (solo si necesitas abrirlo en Excel - tamaño muy grande)
# df_final.to_csv('prendas_con_embeddings_final.csv', index=False, sep=';')
# print("   ✓ Guardado como 'prendas_con_embeddings_final.csv'")


# ============================================
# 6. EJEMPLO DE USO DEL DATAFRAME FINAL
# ============================================

print("\n🎯 Ejemplo de acceso a embeddings:")

# Acceder al embedding de una prenda específica
id_ejemplo = df_final.iloc[0]['ID']
embedding_ejemplo = df_final[df_final['ID'] == id_ejemplo]['embedding_tabla'].iloc[0]

print(f"   Prenda ID: {id_ejemplo}")
print(f"   Categoría: {df_final[df_final['ID'] == id_ejemplo]['category'].iloc[0]}")
print(f"   Color: {df_final[df_final['ID'] == id_ejemplo]['color_name'].iloc[0]}")
print(f"   Embedding (primeros 5 valores): {embedding_ejemplo[:5]}")

print("\n✨ ¡Proceso completado! Tu DataFrame ahora tiene todas las columnas originales + embeddings")

In [ ]:
df_final['embedding_tabla']